In [1]:
library('clusterProfiler')
library(org.Mm.eg.db)
library(ggplot2)



clusterProfiler v4.14.6 Learn more at https://yulab-smu.top/contribution-knowledge-mining/

Please cite:

T Wu, E Hu, S Xu, M Chen, P Guo, Z Dai, T Feng, L Zhou, W Tang, L Zhan,
X Fu, S Liu, X Bo, and G Yu. clusterProfiler 4.0: A universal
enrichment tool for interpreting omics data. The Innovation. 2021,
2(3):100141


Attaching package: ‘clusterProfiler’


The following object is masked from ‘package:stats’:

    filter


Loading required package: AnnotationDbi

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Positi

In [2]:
df.deg <- read.csv("/data2st1/junyi/output/atac0526/dar/region_nt/mementob_dar_noccur.csv")

In [3]:
head(df.deg)

,names,tx,de_coef,de_se,de_pval,dv_coef,dv_se,dv_pval,ctname,region,⋯,annotation_x,distance,de_pval_adj,primary_region,secondary_region,ctname_con,regulation,celltype.L1,Neu_subtype,X.Neuron_type
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,⋯,<chr>,<int>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
1,chr2:98662628-98663129,stim,0.2976482,0.04024950,1.275518e-13,-0.12640698,0.10784833,0.23485742,Immune,PFC,⋯,genebody,0,3.038415e-12,exon,LINE,PFC_Immune_Up,Up,Immune,PFC_GABA/PFC_Glut,2
2,chrX:143482812-143483313,stim,0.3670956,0.04964635,1.308574e-13,0.01050427,0.12550683,0.99252672,Immune,PFC,⋯,genebody,35278,3.114843e-12,distal,LINE,PFC_Immune_Up,Up,Immune,PFC_GABA/PFC_Glut,2
3,chr6:103648949-103649450,stim,0.2977530,0.04019834,1.474284e-13,-0.14411598,0.09456434,0.11074935,Immune,PFC,⋯,genebody,0,3.496496e-12,intron,LINE,PFC_Immune_Up,Up,Immune,PFC_GABA/PFC_Glut,2
4,chr2:98662045-98662546,stim,0.3140902,0.04290548,2.253055e-13,-0.46967550,0.25192550,0.07409584,Immune,PFC,⋯,genebody,0,5.274371e-12,exon,repeat,PFC_Immune_Up,Up,Immune,PFC_GABA/PFC_Glut,2
5,chr12:20815539-20816040,stim,0.6797508,0.09814741,5.078591e-12,-0.04539069,0.22658840,0.72850774,Immune,PFC,⋯,genebody,0,1.072659e-10,exon,cpg,PFC_Immune_Up,Up,Immune,,NA
6,chr19:40723493-40723994,stim,0.2903624,0.04177067,5.362831e-12,-0.09695953,0.15028324,0.55523606,Immune,PFC,⋯,genebody,0,1.130513e-10,intron,LINE,PFC_Immune_Up,Up,Immune,HIP_Glut/HIP_GABA/PFC_GABA/AMY_Glut/PFC_Glut,5


In [4]:
# 获取所有唯一的ct_name和condition组合
ct.names <- c("AMY_Glut", "AMY_GABA", "HIP_Glut", "HIP_GABA", "PFC_Glut", "PFC_GABA")
conditions <- c("MC", "MW")

outfolder <- "/data2st1/junyi/output/atac0526/dar/region_nt/"


In [7]:
subset_df$gene

[1] "Gm31045"       "Adtrp"         "Ywhaq"         "Gm10165"      
   [5] "Serpina3n"     "Khsrp"         "Oaz1"          "Gm47517"      
   [9] "Hjurp"         "Nceh1"         "Apip"          "Nabp1"        
  [13] "Cks2"          "Pdia4"         "Noct"          "Ap1s3"        
  [17] "Tmem18"        "Gm50340"       "Osbp"          "Ccnk"         
  [21] "Pnn"           "Epha7"         "Gm10076"       "Syn2"         
  [25] "Oxr1"          "Gm14254"       "Odc1"          "Fam171a1"     
  [29] "Ociad2"        "BC030867"      "Ubl4b"         "Brms1l"       
  [33] "Spg11"         "Arhgap5"       "5330413P13Rik" "Csmd1"        
  [37] "Dnajb11"       "Gna12"         "4932443L11Rik" "Ngdn"         
  [41] "Ogfod1"        "Tasor2"        "Gm13296"       "Gfod1"        
  [45] "Gm25888"       "Erich3"        "Zmynd8"        "Gm24690"      
  [49] "Dlgap1"        "Rab10os"       "Gm33045"       "Peak1"        
  [53] "Rab3b"         "Gad1"          "4930481A15Rik" "Gm47863"      
  [57] "2900076G11Rik" "Gm13338"       "Rab27b"        "Gm31107"      
  [61] "Scfd1"         "3110056K07Rik" "Derl2"         "Zfp444"       
  [65] "Mical2"        "Ensa"          "Shank2"        "Dgkg"         
  [69] "Cmtr2"         "Pde4d"         "Gm50382"       "Bmt2"         
  [73] "Mboat2"        "Ppp2r1a"       "Pebp4"         "Scn8a"        
  [77] "Schip1"        "Gm20388"       "4930404H11Rik" "Rps6ka5"      
  [81] "Gm19090"       "Gm45062"       "Zfp846"        "Prkn"         
  [85] "R3hcc1"        "Rab10"         "Cttnbp2nl"     "1700064H15Rik"
  [89] "1700012D14Rik" "Gm23649"       "Sdk2"          "Sf3b6"        
  [93] "AC124502.1"    "Col18a1"       "Ccnf"          "1110020A21Rik"
  [97] "1700084C06Rik" "Gm48296"       "5430402P08Rik" "Gm37240"      
 [101] "Tedc1"         "Rere"          "Ube2n"         "Camk2n1"      
 [105] "Numb"          "Dicer1"        "Gm44851"       "Fos"          
 [109] "Eif2ak3"       "Mdga2"         "Hnrnpc"        "2010013B24Rik"
 [113] "Hsd17b12"      "Snx7"          "Rasl11a"       "Isca2"        
 [117] "Vac14"         "D330050G23Rik" "8030442B05Rik" "Smarcal1"     
 [121] "Camta1"        "Gm32051"       "Pin1"          "Gm27944"      
 [125] "Gm40761"       "Neu2"          "Gng7"          "Eml6"         
 [129] "Hip1"          "Ruvbl1"        "Gm5461"        "Kcnk12"       
 [133] "Cntnap5c"      "Mylip"         "Vkorc1l1"      "Gm37240"      
 [137] "Xpo1"          "B530045E10Rik" "Gm5112"        "Gm48021"      
 [141] "Larp4b"        "4833405L11Rik" "Brip1"         "Fam135b"      
 [145] "Gm18907"       "Ralb"          "Nt5e"          "Garem1"       
 [149] "Cfap36"        "Tial1"         "Pik3r3"        "Gars"         
 [153] "Msmo1"         "Crlf2"         "Timm21"        "Gm8784"       
 [157] "Capn2"         "Gm36539"       "Kctd9"         "Kif28"        
 [161] "Mir670hg"      "Tlr5"          "Coq8a"         "Nans"         
 [165] "Rcl1"          "Zmiz1os1"      "Meak7"         "Lrriq3"       
 [169] "Cacybp"        "1810007C17Rik" "Gm14232"       "Gm6159"       
 [173] "Nsmce2"        "Gm26812"       "Kansl2"        "M6pr"         
 [177] "Srgap3"        "Stil"          "Lamb3"         "Dcaf15"       
 [181] "Rbl2"          "Uggt2"         "Gm37843"       "Hykk"         
 [185] "Tamm41"        "Tars"          "Ggct"          "Stard4"       
 [189] "Ubtd1"         "Samd4"         "Fra10ac1"      "Aldh18a1"     
 [193] "Jazf1"         "Lncbate10"     "Rab2b"         "Foxg1"        
 [197] "Gm25574"       "Cep85"         "D430001F17Rik" "Ntrk2"        
 [201] "Crlf2"         "Cobl"          "Gls"           "Bub1b"        
 [205] "BC006965"      "Scgn"          "1700012D14Rik" "D730003K21Rik"
 [209] "Stip1"         "Trim24"        "Gm44066"       "Ube2t"        
 [213] "Traf5"         "Cdh4"          "Nck1"          "E430024I08Rik"
 [217] "Kif13b"        "Map4"          "Creb3l2"       "Tns1"         
 [221] "Gm13377"       "Ero1lb"        "Gm48547"       "Mmgt2"        
 [225] "Ccdc61"        "Dro

In [8]:
# 转换ID（Symbol → Entrez）

# ctname <- "PFC_Glut"
# condition <- "MW"

for (ctname in ct.names) {
  for (condition in conditions) {
    # 筛选数据
    tryCatch(
      {
        print(paste("Processing:", ctname, condition))

        subset_df <- df.deg[df.deg$ctname == ctname & df.deg$condition == condition, ]

        print(paste("Number of genes:", nrow(subset_df)))

        gene_entrez <- bitr(unique(subset_df$gene), 
                            fromType = "SYMBOL", 
                            toType = "ENTREZID", 
                            OrgDb = "org.Mm.eg.db")
                            
        entrez_ids <- gene_entrez$ENTREZID
        ego <- enrichGO(gene = entrez_ids,
                        OrgDb = org.Mm.eg.db,
                        keyType = "ENTREZID",
                        ont = "ALL",  # "BP", "MF", "CC" 或 "ALL"
                        pvalueCutoff = 0.05,
                        qvalueCutoff = 0.05,
                        readable = TRUE)  # 结果中显示基因符号

        # 查看结果
        write.csv(ego, file = paste(outfolder, "GO_enrichment_memento",ctname,condition,".csv",sep = "_"), row.names = FALSE)
        p <- dotplot(ego, showCategory = 30)
        ggsave(paste(outfolder, "GO_enrichment_memento_dot",ctname,condition,".pdf",sep = "_"), plot = p, width = 12, height = 15)  # 宽度12英寸，高度15英寸

        p<-barplot(ego, showCategory = 30)  # 条形图
        ggsave(paste(outfolder, "GO_enrichment_memento_bar",ctname,condition,".pdf",sep = "_"), plot = p, width = 12, height = 15)  # 宽度12英寸，高度15英寸

        kk <- enrichKEGG(gene = entrez_ids,
                        organism = "mmu",  # 小鼠KEGG代码
                        pvalueCutoff = 0.05,
                        qvalueCutoff = 0.05)

        # 可视化
        p<-dotplot(kk, showCategory = 30)
        write.csv(kk, file = paste(outfolder, "KEGG_enrichment_memento",ctname,condition,".csv",sep = "_"), row.names = FALSE)
        ggsave(paste(outfolder, "kegg_enrichment_memento_dot",ctname,condition,".pdf",sep = "_"), plot = p, width = 12, height = 15)  # 宽度12英寸，高度15英寸
      }, 
      error = function(e) {
        print(paste("Error in subsetting data for", ctname, condition))
    })

  }
}

[1] "Processing: AMY_Glut MC"
[1] "Number of genes: 5314"


'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(unique(subset_df$gene), fromType = "SYMBOL", toType = "ENTREZID", :
“8.11% of input gene IDs are fail to map...”
Reading KEGG annotation online: "https://rest.kegg.jp/link/mmu/pathway"...

Reading KEGG annotation online: "https://rest.kegg.jp/list/pathway/mmu"...



[1] "Processing: AMY_Glut MW"
[1] "Number of genes: 9041"


'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(unique(subset_df$gene), fromType = "SYMBOL", toType = "ENTREZID", :
“14.03% of input gene IDs are fail to map...”


[1] "Processing: AMY_GABA MC"
[1] "Number of genes: 6808"


...

'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(unique(subset_df$gene), fromType = "SYMBOL", toType = "ENTREZID", :
“12.49% of input gene IDs are fail to map...”


[1] "Processing: PFC_GABA MC"
[1] "Number of genes: 321"


'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(unique(subset_df$gene), fromType = "SYMBOL", toType = "ENTREZID", :
“10.49% of input gene IDs are fail to map...”


[1] "Processing: PFC_GABA MW"
[1] "Number of genes: 9792"


'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(unique(subset_df$gene), fromType = "SYMBOL", toType = "ENTREZID", :
“8.8% of input gene IDs are fail to map...”


In [ ]:
combined_go <- list()
combined_kegg <- list()
for (ctname in ct.names) {
  for (condition in conditions) {
    # 筛选数据
    tryCatch(
      {
        print(paste("Processing:", ctname, condition))

        subset_df <- df.deg[df.deg$ctname == ctname & df.deg$condition == condition, ]

        print(paste("Number of genes:", nrow(subset_df)))

        gene_entrez <- bitr(unique(subset_df$gene), 
                            fromType = "SYMBOL", 
                            toType = "ENTREZID", 
                            OrgDb = "org.Mm.eg.db")
                            
        entrez_ids <- gene_entrez$ENTREZID
        ego <- enrichGO(gene = entrez_ids,
                        OrgDb = org.Mm.eg.db,
                        keyType = "ENTREZID",
                        ont = "ALL",  # "BP", "MF", "CC" 或 "ALL"
                        pvalueCutoff = 0.05,
                        qvalueCutoff = 0.05,
                        readable = TRUE)  # 结果中显示基因符号
          
        # combined_go[[paste(ctname, condition, sep = "_")]] <- ego
        combined_go[[paste(ctname, condition, sep = "_")]] <- ego

        # 查看结果
        # write.csv(ego, file = paste(outfolder, "GO_enrichment_memento",ctname,condition,".csv",sep = "_"), row.names = FALSE)
        # p <- dotplot(ego, showCategory = 30)
        # ggsave(paste(outfolder, "GO_enrichment_memento_dot",ctname,condition,".pdf",sep = "_"), plot = p, width = 12, height = 15)  # 宽度12英寸，高度15英寸

        # p<-barplot(ego, showCategory = 30)  # 条形图
        # ggsave(paste(outfolder, "GO_enrichment_memento_bar",ctname,condition,".pdf",sep = "_"), plot = p, width = 12, height = 15)  # 宽度12英寸，高度15英寸

        kk <- enrichKEGG(gene = entrez_ids,
                        organism = "mmu",  # 小鼠KEGG代码
                        pvalueCutoff = 0.05,
                        qvalueCutoff = 0.05)

        combined_kegg[[paste(ctname, condition, sep = "_")]] <- kk

        # # 可视化
        # p<-dotplot(kk, showCategory = 30)
        # write.csv(kk, file = paste(outfolder, "KEGG_enrichment_memento",ctname,condition,".csv",sep = "_"), row.names = FALSE)
        # ggsave(paste(outfolder, "kegg_enrichment_memento_dot",ctname,condition,".pdf",sep = "_"), plot = p, width = 12, height = 15)  # 宽度12英寸，高度15英寸
      }, 
      error = function(e) {
        print(paste("Error in subsetting data for", ctname, condition))
    })

  }
}

[1] "Processing: AMY_Glut MC"
[1] "Number of genes: 5354"


'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(unique(subset_df$gene_name), fromType = "SYMBOL", toType = "ENTREZID", :
“8.04% of input gene IDs are fail to map...”
Reading KEGG annotation online: "https://rest.kegg.jp/link/mmu/pathway"...

Reading KEGG annotation online: "https://rest.kegg.jp/list/pathway/mmu"...



[1] "Processing: AMY_Glut MW"
[1] "Number of genes: 9056"


'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(unique(subset_df$gene_name), fromType = "SYMBOL", toType = "ENTREZID", :
“13.46% of input gene IDs are fail to map...”


[1] "Processing: AMY_GABA MC"
[1] "Number of genes: 6850"


...

'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(unique(subset_df$gene_name), fromType = "SYMBOL", toType = "ENTREZID", :
“11.89% of input gene IDs are fail to map...”


[1] "Processing: PFC_GABA MC"
[1] "Number of genes: 313"


'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(unique(subset_df$gene_name), fromType = "SYMBOL", toType = "ENTREZID", :
“9.75% of input gene IDs are fail to map...”


[1] "Processing: PFC_GABA MW"
[1] "Number of genes: 9884"


'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(unique(subset_df$gene_name), fromType = "SYMBOL", toType = "ENTREZID", :
“8.49% of input gene IDs are fail to map...”


In [40]:
gene_lists <- list(
  AMY_Glut_MC = combined_go$AMY_Glut_MC@gene,  # Vector of gene IDs (e.g., ENSEMBL)
  AMY_Glut_MW = combined_go$AMY_Glut_MW@gene,
  AMY_GABA_MC = combined_go$AMY_GABA_MC@gene,
  AMY_GABA_MW = combined_go$AMY_GABA_MW@gene,
  HIP_Glut_MC = combined_go$HIP_Glut_MC@gene,
  HIP_Glut_MW = combined_go$HIP_Glut_MW@gene,
  HIP_GABA_MC = combined_go$HIP_GABA_MC@gene,
  HIP_GABA_MW = combined_go$HIP_GABA_MW@gene,
  PFC_Glut_MC = combined_go$PFC_Glut_MC@gene,
  PFC_Glut_MW = combined_go$PFC_Glut_MW@gene,
  PFC_GABA_MC = combined_go$PFC_GABA_MC@gene,
  PFC_GABA_MW = combined_go$PFC_GABA_MW@gene
)


In [41]:
comp_results <- compareCluster(
  geneClusters = gene_lists,
  fun = "enrichGO",
  OrgDb = "org.Mm.eg.db",  # Mouse database
  keyType = "ENTREZID",      # Or "SYMBOL"
  ont = "BP"               # Biological Process
)


In [43]:
pdit<-dotplot(comp_results, 
        showCategory = 10, 
        split = NULL, 
        font.size = 8,
        title = "GO Comparison Across Brain Regions")
ggsave(paste(outfolder, "GO_enrichment_memento_dot_comparison",ctname,condition,".pdf",sep = "_"), plot = pdit, width = 20, height = 20)  # 宽度12英寸，高度15英寸

In [4]:
ctname <- "PFC_Glut"
condition <- "MW"
# 筛选数据

subset_df <- df.deg[df.deg$ctname == ctname & df.deg$condition == condition, ]

print(paste("Number of genes:", nrow(subset_df)))

gene_entrez <- bitr(unique(subset_df$gene_name), 
                    fromType = "SYMBOL", 
                    toType = "ENTREZID", 
                    OrgDb = "org.Mm.eg.db")
                    
entrez_ids <- gene_entrez$ENTREZID
ego <- enrichGO(gene = entrez_ids,
                OrgDb = org.Mm.eg.db,
                keyType = "ENTREZID",
                ont = "ALL",  # "BP", "MF", "CC" 或 "ALL"
                pvalueCutoff = 0.05,
                qvalueCutoff = 0.05,
                readable = TRUE)  # 结果中显示基因符号

[1] "Number of genes: 35404"


'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(unique(subset_df$gene_name), fromType = "SYMBOL", toType = "ENTREZID", :
“11.89% of input gene IDs are fail to map...”


In [8]:
library(enrichplot)

# edo <- pairwise_termsim(ego)
# p1<-emapplot(edo)
# p2 <- emapplot(edo, cex_category=1.5)
# p3 <- emapplot(edo, layout="kk")
# p4<-emapplot(edo, cex_category=1.5,layout="kk") 
# pp<-cowplot::plot_grid(p1, p2, p3, p4, ncol=2, labels=LETTERS[1:4])
ggsave(paste(outfolder, "GO_enrichment_memento_emap",ctname,condition,".pdf",sep = "_"), plot = pp, width = 20, height = 20)  # 宽度12英寸，高度15英寸


In [9]:
edo

#
# over-representation test
#
#...@organism 	 Mus musculus 
#...@ontology 	 GOALL 
#...@keytype 	 ENTREZID 
#...@gene 	 chr [1:14262] "244310" "214058" "217390" "665731" "244682" "115488866" ...
#...pvalues adjusted by 'BH' with cutoff <0.05 
#...4930 enriched terms found
'data.frame':	4930 obs. of  10 variables:
 $ ONTOLOGY   : chr  "BP" "BP" "BP" "BP" ...
 $ ID         : chr  "GO:0043161" "GO:0022613" "GO:0034470" "GO:0044772" ...
 $ Description: chr  "proteasome-mediated ubiquitin-dependent protein catabolic process" "ribonucleoprotein complex biogenesis" "ncRNA processing" "mitotic cell cycle phase transition" ...
 $ GeneRatio  : chr  "352/12099" "339/12099" "317/12099" "339/12099" ...
 $ BgRatio    : chr  "451/28891" "440/28891" "413/28891" "454/28891" ...
 $ pvalue     : num  4.09e-56 1.01e-51 8.39e-48 2.50e-46 4.57e-46 ...
 $ p.adjust   : num  2.66e-52 3.29e-48 1.81e-44 4.05e-43 5.94e-43 ...
 $ qvalue     : num  1.10e-52 1.37e-48 7.55e-45 1.69e-43 2.47e-43 ...
 $ geneID     : c

In [2]:
flist <- Sys.glob(file.path("/data2st1/junyi/output/atac0526/dar/region_nt/_GO*enrichment*memento*.csv"))


In [3]:
flist

[1] "/data2st1/junyi/output/atac0526/dar/region_nt/_GO_enrichment_memento_AMY_GABA_MC_.csv"
 [2] "/data2st1/junyi/output/atac0526/dar/region_nt/_GO_enrichment_memento_AMY_GABA_MW_.csv"
 [3] "/data2st1/junyi/output/atac0526/dar/region_nt/_GO_enrichment_memento_AMY_Glut_MC_.csv"
 [4] "/data2st1/junyi/output/atac0526/dar/region_nt/_GO_enrichment_memento_AMY_Glut_MW_.csv"
 [5] "/data2st1/junyi/output/atac0526/dar/region_nt/_GO_enrichment_memento_HIP_GABA_MC_.csv"
 [6] "/data2st1/junyi/output/atac0526/dar/region_nt/_GO_enrichment_memento_HIP_GABA_MW_.csv"
 [7] "/data2st1/junyi/output/atac0526/dar/region_nt/_GO_enrichment_memento_HIP_Glut_MC_.csv"
 [8] "/data2st1/junyi/output/atac0526/dar/region_nt/_GO_enrichment_memento_HIP_Glut_MW_.csv"
 [9] "/data2st1/junyi/output/atac0526/dar/region_nt/_GO_enrichment_memento_PFC_GABA_MC_.csv"
[10] "/data2st1/junyi/output/atac0526/dar/region_nt/_GO_enrichment_memento_PFC_GABA_MW_.csv"
[11] "/data2st1/junyi/output/atac0526/dar/region_nt/_GO_enrichment_memento_PFC_Glut_MC_.csv"
[12] "/data2st1/junyi/output/atac0526/dar/region_nt/_GO_enrichment_memento_PFC_Glut_MW_.csv"

In [70]:
all_data <- data.frame()
for (f in flist) {
  print(paste("Processing file:", f))

  # 读取数据
  df <- read.csv(f, header = TRUE)
  
  # 提取ctname和condition
  parts <- strsplit(basename(f), "_")[[1]]
  ctname <- paste0(parts[5], "_", parts[6])
  condition <- parts[7]
  region <- parts[5]
  
  df$ctname <- ctname
  df$condition <- condition
  df$region <- region
  df$ct<-parts[6]

  dfh <- head(df,n = 10)

  # 创建输出文件名
  #output_file <- paste0("/data2st1/junyi/output/atac0416/dar/region_nt/", ctname, "_", condition, "_enrichment.csv")
  
  # 写入文件
  #write.csv(df, file = output_file, row.names = FALSE)
  # Concatenate the data
  all_data <- rbind(all_data, dfh)
}


[1] "Processing file: /data2st1/junyi/output/atac0526/dar/region_nt/_GO_enrichment_memento_AMY_GABA_MC_.csv"
[1] "Processing file: /data2st1/junyi/output/atac0526/dar/region_nt/_GO_enrichment_memento_AMY_GABA_MW_.csv"
[1] "Processing file: /data2st1/junyi/output/atac0526/dar/region_nt/_GO_enrichment_memento_AMY_Glut_MC_.csv"
[1] "Processing file: /data2st1/junyi/output/atac0526/dar/region_nt/_GO_enrichment_memento_AMY_Glut_MW_.csv"
[1] "Processing file: /data2st1/junyi/output/atac0526/dar/region_nt/_GO_enrichment_memento_HIP_GABA_MC_.csv"
[1] "Processing file: /data2st1/junyi/output/atac0526/dar/region_nt/_GO_enrichment_memento_HIP_GABA_MW_.csv"
[1] "Processing file: /data2st1/junyi/output/atac0526/dar/region_nt/_GO_enrichment_memento_HIP_Glut_MC_.csv"
[1] "Processing file: /data2st1/junyi/output/atac0526/dar/region_nt/_GO_enrichment_memento_HIP_Glut_MW_.csv"
[1] "Processing file: /data2st1/junyi/output/atac0526/dar/region_nt/_GO_enrichment_memento_PFC_GABA_MC_.csv"
[1] "Processing fil

In [71]:
head(df)

,ONTOLOGY,ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count,ctname,condition,region,ct
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>
1,BP,GO:0043161,proteasome-mediated ubiquitin-dependent protein catabolic process,361/12302,451/28891,1.667298e-60,1.082243e-56,4.635089e-57,Man1c1/Man1a/Rnf144b/Eif2ak3/Xpo1/Prkn/Fhit/Ubxn10/Ubxn11/Trim2/Akirin2/Prickle1/Sec61b/Fbxl17/Trib1/Csnk1a1/Trim45/Nedd4l/Klhl3/Ubr2/Edem3/Gna12/Mapk9/Rnf144a/Trib2/Zer1/Eif3h/Siah3/Hsp90b1/Sh3rf1/Dtx4/Znrf1/Fem1c/Rnf5/Ube2d3/Rad23b/Kif14/Trim3/Ccnf/Derl2/Psmb5/Uchl5/Wwtr1/Fbxw8/Znrf4/Fbxl13/Rybp/Gsk3b/Ubr1/Clu/Cdc26/Stt3b/Fbxo48/Rhobtb3/Rnf216/Cd2ap/Klhdc10/Nkd2/Wwp1/Birc2/Nsfl1c/Skp2/Ube2n/Canx/Anapc4/Sh3rf2/Ambra1/Ddrgk1/Fbxo2/Ubxn2b/Sh3rf3/Mdm2/Wfs1/Rnf121/Smurf1/Psmc2/Ube2w/Znrf2/Trim9/Fbxo45/Map1a/Ubxn2a/Ube4b/Tnfaip1/Vcp/Pja2/Clock/Clec16a/Ufd1/Ltn1/Psmd4/Fem1a/Cdc23/Nhlrc1/Fbxl21/Fbxl19/Herc2/Rnf34/Rbx1/Cul4a/Herpud1/Ube2c/Fbxw11/Amn1/Rnf4/Armc8/Ubr3/Chfr/Cbfa2t3/Ppp2r5c/Sirt2/Aup1/Aurka/Arrb2/Smurf2/Amfr/Gipc1/Rack1/Atg7/Ufl1/Psma5/Plk1/Rmnd5a/Fbxo39/Psmb3/Selenos/Pcnp/Derl1/Cul2/Ube2s/Dcaf13/Spsb3/Cdc34/Trip4/Man1b1/Otud5/Peli1/Fbxo4/Hspbp1/Anapc10/Edem2/Svip/Derl3/Klhl20/Psmc3/Dda1/Fem1b/Zyg11b/Kbtbd7/Ube2d1/Plk2/Gid4/Fbxl4/Araf/Tmem67/Syvn1/Ascc3/Gsk3a/Fbxo17/Btrc/Ercc8/Os9/Spop/Plaa/Fbxo33/Psmd8/Csnk1e/Rnf126/Ube2j2/Trpc4ap/Nploc4/Ccdc47/Psmc6/Gid8/Hspa5/Rpl11/Smarcc1/Spsb1/Maea/Siah1b/Anapc5/Jkamp/Psmd14/Dtl/Usp19/Siah2/Bbs7/Tmub2/Zfp418/Tbl1xr1/Fbxw7/Spsb2/Pml/Epm2a/Ubxn7/Lrrk2/Psmf1/Desi1/Kctd17/Pias1/Plk3/Mtm1/Ube2g1/Pcbp2/Arih1/Zfp598/Fbxo27/Fbxl22/Psmc4/Psen2/Ddb1/Atxn3/Caml/Hectd1/Rmnd5b/Kbtbd6/Dnajc10/Axin2/Rnf217/Mta1/Ecrg4/Apc/Sumo2/Fbxl3/Socs4/Rnf19b/Ubxn8/Cdc20b/Anapc1/Ubqln1/Fbxo31/Sirt6/Ube2k/Fbxw5/Ube4a/Wwp2/Spopl/Gclc/Fbxl8/Snhg15/Psen1/Usp7/Arrb1/Kctd5/Edem1/N4bp1/Kctd2/Ubxn4/Erlin2/Itch/Axin1/Klhl42/Ubxn1/Usp14/Ube2a/Rnf103/Rnf14/Psmc1/Psmd2/Nop53/Ube2v2/Ctnnb1/Klhdc2/Appbp2/Rbck1/Nub1/Faf2/Dcaf12/Asb2/Rnf19a/Fbxl20/Socs5/Ttc36/Trim72/Zfand2b/Dnajb2/Nrros/Ccar2/Ube2h/Mapk8/Fbxw4/Fbxo44/Trim39/Psmd1/Trim13/Clgn/Sgta/Psmd7/Cop1/Cdc20/Kat5/Dcaf11/Arih2/Sumo1/Klhdc3/Klhl22/Hspa1b/Cul3/Pmp22/Trip12/Styx/Tlk2/Psmd6/Ube2g2/Fzr1/Nhlrc3/Ube2b/Fbxo22/Ube2j1/Fbxl2/Nemf/Ogt/Crbn/Sharpin/Park7/Bag6/Rnf186/Zfand2a/Usp9x/Ankrd9/Phf20l1/Ubqln4/Rnf187/Dmac2/Apc2/Ecpas/Tmub1/Ascc2/Agap3/Rad23a/Zswim8/Ubqln2/Wdr26/Kctd13/Faf1/Rnf10/Fbxl16/Fbxl15/Ppp2cb/Fbxo6/Klhl40/Ube3a/Cdc16/Klhdc1/Gabarap/Csnk1d/Fbxl18/Wac/Akt1/Ddit3/Man1a2/Taf9/Sirt1/Topors/Stub1/Rffl/Erlin1/Usp5/Fbxl14/Cul5/Foxred2,361,PFC_Glut,MW,PFC,Glut
2,BP,GO:0022613,ribonucleoprotein complex biogenesis,347/12302,440/28891,2.843142e-55,9.227418e-52,3.951968e-52,Kat2b/Shq1/Xpo1/Lsm4/Heatr1/Celf2/Prmt5/Urb1/Ddx20/Serbp1/Rps14/Ago2/Nhp2/Ngdn/Utp4/Celf4/Ddx10/Celf6/Rbis/Tarbp2/Srpk1/Tsc1/Pa2g4/Cdc73/Npm1/Bms1/Utp20/Nufip1/Eif2a/Mtrex/Heatr3/Rpusd1/Xrcc5/Krr1/Usp36/Ruvbl1/Rps13/Wdr43/Pop5/Rps6/Nsun3/Abt1/Rps12/Ddx52/Rcl1/Eif5/Tssc4/Ythdf2/Chd7/Gemin6-ps/Celf1/Wdr75/Nol8/Nol6/Rpp40/Mettl17/Ddx18/Zfp622/Rps7/Sf3a3/Mdn1/Rrs1/Prpf18/Ythdc1/Naf1/Dimt1/Wdr36/Khdc4/Gemin8/Ddx17/Drosha/Imp3/Pelp1/Gemin2/Rps5/Aatf/Nsun4/Cul4a/LTO1/Suv39h1/Nip7/Ipo9/Rrp1b/Prpf8/Rbm5/Rps11/Wbp11/Utp3/Mettl16/Atr/Pten/Eif2s2/Eri3/Mrpl10/Nol11/Fastkd2/Rps21/Pin4/Rps24/Eif2s3x/Celf5/Nsa2/Rps19/Dcaf13/Rpl35a/Rpsa/Rexo4/Efl1/Utp18/Gtpbp4/Utp11/Srfbp1/Ckap5/Mak16/Riok1/Tma16/Rpl26/Dicer1/Rpf1/Clp1/Gtpbp10/Ncl/Gemin5/Nmd3/Malsu1/Eri1/Lsm6/Prpf6/Rpl23a/Clns1a/Riok2/Dhx37/Denr/Strap/Utp15/Ftsj3/Lsg1/Ybey/Rpl24/Eif3a/Rpl11/Ercc2/Gnl2/Utp14a/Dkc1/Luc7l2/Rps15a/Puf60/Pwp2/Rrp12/Nvl/Tfb2m/Las1l/Srsf10/Rpl7a/Nop58/Bop1/Nom1/Isy1/Gemin6/Sf3b1/Gtf2h5/Wdr12/Rbm34/Riok3/Eif2d/Fcf1/Rpl10/Snrpf/Usp16/Gcfc2/Setx/Pdcd11/Pak1ip1/Rsl24d1/Sf3a1/Rplp0/Snrpb/Ltv1/Isg20l2/Surf6/Riox1/Znhit6/Rpusd4/Rpl7/Fdxacb1/Utp23/Ddx28/Ddx56/Rps16/Luc7l3/Snrpe/Psip1/Scaf11/Prpf39/Fbl/Nop56/Prmt7/Prpf31/Coil/Exosc5/Exosc2/Adar/Bysl/Crnkl1/Gnl3l/Utp6/Ppan/Nup88/Cul4b/Sde2/Snrnp200/Rbm10/Mybbp1a/Fbll1/Srp

In [49]:
ego<-read.csv(f, header = TRUE)

In [50]:
p<-barplot(ego, showCategory = 30)  # 条形图


ERROR: Error in barplot.default(ego, showCategory = 30): 'height' must be a vector or a matrix


In [72]:
library(dplyr)

# Identify pathways with mixed regulation in the same CT
mixed_pathways <- all_data %>%
  group_by(ctname, Description) %>%
  summarise(has_up = any(condition == "MC"),
            has_down = any(condition == "MW")) 

`summarise()` has grouped output by 'ctname'. You can override using the
`.groups` argument.


In [73]:
# Filter for pathways that have both up and down regulation
mixed_pathways <- mixed_pathways %>%
  filter(has_up & has_down) %>%
  select(ctname, Description)

In [74]:
mixed_pathways

ctname,Description
<chr>,<chr>
AMY_Glut,dendrite development
HIP_Glut,dendrite development


In [75]:
df_processed <- all_data %>%
  mutate(
    p.adjust = ifelse(is.na(p.adjust), 1, p.adjust), # Fill NA with p=1 (least significant)
    log_padj = -log10(p.adjust),
    # Create regulation-specific score (positive for Up, negative for Down)
    specificity_score = ifelse(condition == "MC", log_padj, -log_padj)
  ) %>%
  group_by(region, ct, condition) %>%
  # Rank pathways within each group by absolute specificity
  mutate(
    Rank = rank(-abs(specificity_score), ties.method = "first")
  ) %>%
  ungroup()


In [77]:
p<-ggplot(df_processed, aes(
  x = condition, 
  y = reorder(Description, specificity_score), # Order by specificity
  fill = specificity_score  # Use regulation-aware score
)) +
  geom_tile(color = "gray80", linewidth = 0.3) +
  facet_grid(region ~ ct, scales = "free", space = "free") +
  scale_fill_gradient2(
    low = "blue", mid = "white", high = "red", # Down/Up color scale
    midpoint = 0,
    limits = c(-max(abs(df_processed$specificity_score)), 
              max(abs(df_processed$specificity_score))),
    name = "Regulation\nSpecificity"
  ) +
  labs(
    x = "Cell Type", 
    y = "GO Pathway",
    title = "Top GO Pathways by Regulation Specificity"
  ) +
  theme_minimal(base_size = 12) +
  theme(
    axis.text.x = element_text(angle = 45, hjust = 1, vjust = 1),
    panel.grid = element_blank(),
    strip.text = element_text(face = "bold"),
    plot.title = element_text(hjust = 0.5)
  )
ggsave("/data2st1/junyi/output/atac0526/dar/region_nt/all_GO_enrichment_memento_2.pdf",p, width = 8, height = 15, dpi = 300)


In [62]:
p<-ggplot(all_data, aes(x = condition, y = Description, fill = -log10(p.adjust))) +
  geom_tile() +
  facet_grid(region ~ ct, scales = "free", space = "free") +
  scale_fill_gradient(low = "blue", high = "red") +
  labs(x = "Cell Type", y = "GO Pathway", fill = "-log10(p.adjust)") +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1))
ggsave("/data2st1/junyi/output/atac0526/dar/region_nt/all_GO_enrichment_memento.pdf",p, width = 8, height = 9, dpi = 300)


In [12]:
write.csv(all_data, file = "/data2st1/junyi/output/atac0526/dar/region_nt/all_GO_enrichment_memento.csv", row.names = FALSE)